In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
print(os.getcwd())


from pyspark.sql import SparkSession

# Create a Spark session with optimized settings
spark = (
    SparkSession.builder 
    .appName("OptimizedLocalSpark") 
    .config("spark.driver.memory", "8g")        
    .config("spark.executor.memory", "8g")    
    .config("spark.executor.cores", "4")       
    .config("spark.cores.max", "12")           
    .config("spark.sql.shuffle.partitions", "28")  
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") 
    .getOrCreate()
)
sc = spark.sparkContext

H:\pyspark_advanced-coding_interview


In [2]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("TimeZoneConversionExample").getOrCreate()

# Sample Data: Sales records with timestamps in different time zones
data = [
    (1, "2023-01-15 14:00:00", "America/New_York"),
    (2, "2023-01-15 11:00:00", "America/Los_Angeles"),
    (3, "2023-01-15 20:00:00", "Europe/London"),
    (4, "2023-01-16 03:00:00", "Asia/Tokyo"),
    (5, "2023-01-15 19:00:00", "UTC"),
    (6, "2023-01-15 08:00:00", "America/Chicago"),
    (7, "2023-01-15 21:00:00", "Europe/Berlin"),
    (8, "2023-01-16 00:00:00", "Asia/Shanghai"),
    (9, "2023-01-15 13:30:00", "America/New_York"),
    (10, "2023-01-15 17:00:00", "UTC")
]

# Create DataFrame
df = spark.createDataFrame(data, ["event_id", "event_time", "time_zone"])

# Create a Temporary View for Spark SQL
df.createOrReplaceTempView("events")

# Show the Original DataFrame
df.show(truncate=False)


+--------+-------------------+-------------------+
|event_id|event_time         |time_zone          |
+--------+-------------------+-------------------+
|1       |2023-01-15 14:00:00|America/New_York   |
|2       |2023-01-15 11:00:00|America/Los_Angeles|
|3       |2023-01-15 20:00:00|Europe/London      |
|4       |2023-01-16 03:00:00|Asia/Tokyo         |
|5       |2023-01-15 19:00:00|UTC                |
|6       |2023-01-15 08:00:00|America/Chicago    |
|7       |2023-01-15 21:00:00|Europe/Berlin      |
|8       |2023-01-16 00:00:00|Asia/Shanghai      |
|9       |2023-01-15 13:30:00|America/New_York   |
|10      |2023-01-15 17:00:00|UTC                |
+--------+-------------------+-------------------+



In [3]:
#to_utc_timestamp(event_time, time_zone): Converts the local time in the specified time_zone to UTC

res = spark.sql(""" 
SELECT event_id, 
       to_utc_timestamp(event_time, time_zone) AS utc_time
FROM events;


              
                """)
res.show()

+--------+-------------------+
|event_id|           utc_time|
+--------+-------------------+
|       1|2023-01-15 19:00:00|
|       2|2023-01-15 19:00:00|
|       3|2023-01-15 20:00:00|
|       4|2023-01-15 18:00:00|
|       5|2023-01-15 19:00:00|
|       6|2023-01-15 14:00:00|
|       7|2023-01-15 20:00:00|
|       8|2023-01-15 16:00:00|
|       9|2023-01-15 18:30:00|
|      10|2023-01-15 17:00:00|
+--------+-------------------+



In [4]:
#to_utc_timestamp: Converts the local time in the specified time_zone to UTC.
#from_utc_timestamp: Converts the UTC time to the desired time zone (America/New_York)



res1 = spark.sql(""" 
SELECT event_id, 
       from_utc_timestamp(to_utc_timestamp(event_time, time_zone), 'America/New_York') AS event_time_ny
FROM events;


              
                """)
res1.show()

+--------+-------------------+
|event_id|      event_time_ny|
+--------+-------------------+
|       1|2023-01-15 14:00:00|
|       2|2023-01-15 14:00:00|
|       3|2023-01-15 15:00:00|
|       4|2023-01-15 13:00:00|
|       5|2023-01-15 14:00:00|
|       6|2023-01-15 09:00:00|
|       7|2023-01-15 15:00:00|
|       8|2023-01-15 11:00:00|
|       9|2023-01-15 13:30:00|
|      10|2023-01-15 12:00:00|
+--------+-------------------+



In [14]:
spark.conf.set("spark.sql.session.timeZone", "UTC")


In [16]:
# Set the default Spark session time zone to UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")


from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_utc_timestamp, from_utc_timestamp

# Initialize Spark Session
spark = SparkSession.builder.appName("TimeZoneConversionExample").getOrCreate()

# Set the Spark session time zone to UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")

# Create Sample Data
data = [
    (1, "2023-01-15 14:00:00", "America/New_York"),
    (2, "2023-01-15 11:00:00", "America/Los_Angeles"),
    (3, "2023-01-15 20:00:00", "Europe/London"),
    (4, "2023-01-16 03:00:00", "Asia/Tokyo"),
    (5, "2023-01-15 19:00:00", "UTC")
]

# Create DataFrame
df = spark.createDataFrame(data, ["event_id", "event_time", "time_zone"])

# Convert 'event_time' to Proper Timestamp Format
df = df.withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss"))

# Convert to UTC based on provided time zones
df_utc = df.withColumn("utc_time", to_utc_timestamp("event_time", "time_zone"))

# Attempt conversion back to a specific time zone (e.g., America/New_York)
df_ny_time = df_utc.withColumn("ny_time", from_utc_timestamp("utc_time", "America/New_York"))

# Show the Results
df_ny_time.show(truncate=False)



Py4JJavaError: An error occurred while calling o235.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 212) (localhost executor driver): java.time.zone.ZoneRulesException: Unknown time-zone ID: time_zone
	at java.base/java.time.zone.ZoneRulesProvider.getProvider(ZoneRulesProvider.java:279)
	at java.base/java.time.zone.ZoneRulesProvider.getRules(ZoneRulesProvider.java:234)
	at java.base/java.time.ZoneRegion.ofId(ZoneRegion.java:120)
	at java.base/java.time.ZoneId.of(ZoneId.java:408)
	at java.base/java.time.ZoneId.of(ZoneId.java:356)
	at java.base/java.time.ZoneId.of(ZoneId.java:312)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils$.getZoneId(DateTimeUtils.scala:61)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils.getZoneId(DateTimeUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.init(Unknown S

In [8]:
from pyspark.sql.functions import to_timestamp

# Convert the string 'event_time' to a proper timestamp
df = df.withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss"))
df.show()


from pyspark.sql.functions import to_utc_timestamp

# Now convert all event times to UTC
df_utc = df.withColumn("utc_time", to_utc_timestamp("event_time", "time_zone"))

# Show the result
df_utc.show(truncate=False)


from pyspark.sql.functions import from_utc_timestamp

# Convert UTC times to America/New_York time zone
df_ny_time = df_utc.withColumn("ny_time", from_utc_timestamp("utc_time", "America/New_York"))

# Show the result
df_ny_time.show(truncate=False)


+--------+-------------------+-------------------+
|event_id|         event_time|          time_zone|
+--------+-------------------+-------------------+
|       1|2023-01-15 14:00:00|   America/New_York|
|       2|2023-01-15 11:00:00|America/Los_Angeles|
|       3|2023-01-15 20:00:00|      Europe/London|
|       4|2023-01-16 03:00:00|         Asia/Tokyo|
|       5|2023-01-15 19:00:00|                UTC|
|       6|2023-01-15 08:00:00|    America/Chicago|
|       7|2023-01-15 21:00:00|      Europe/Berlin|
|       8|2023-01-16 00:00:00|      Asia/Shanghai|
|       9|2023-01-15 13:30:00|   America/New_York|
|      10|2023-01-15 17:00:00|                UTC|
+--------+-------------------+-------------------+



Py4JJavaError: An error occurred while calling o79.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 207) (localhost executor driver): java.time.zone.ZoneRulesException: Unknown time-zone ID: time_zone
	at java.base/java.time.zone.ZoneRulesProvider.getProvider(ZoneRulesProvider.java:279)
	at java.base/java.time.zone.ZoneRulesProvider.getRules(ZoneRulesProvider.java:234)
	at java.base/java.time.ZoneRegion.ofId(ZoneRegion.java:120)
	at java.base/java.time.ZoneId.of(ZoneId.java:408)
	at java.base/java.time.ZoneId.of(ZoneId.java:356)
	at java.base/java.time.ZoneId.of(ZoneId.java:312)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils$.getZoneId(DateTimeUtils.scala:61)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils.getZoneId(DateTimeUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.init(Unknown So

In [5]:
from pyspark.sql.functions import to_utc_timestamp

# Convert all event times to UTC
df_utc = df.withColumn("utc_time", to_utc_timestamp("event_time", "time_zone"))

# Show the result
df_utc.show(truncate=False)


Py4JJavaError: An error occurred while calling o63.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 60) (localhost executor driver): java.time.zone.ZoneRulesException: Unknown time-zone ID: time_zone
	at java.base/java.time.zone.ZoneRulesProvider.getProvider(ZoneRulesProvider.java:279)
	at java.base/java.time.zone.ZoneRulesProvider.getRules(ZoneRulesProvider.java:234)
	at java.base/java.time.ZoneRegion.ofId(ZoneRegion.java:120)
	at java.base/java.time.ZoneId.of(ZoneId.java:408)
	at java.base/java.time.ZoneId.of(ZoneId.java:356)
	at java.base/java.time.ZoneId.of(ZoneId.java:312)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils$.getZoneId(DateTimeUtils.scala:61)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils.getZoneId(DateTimeUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.init(Unknown Sourc

In [12]:
spark.conf.set("spark.sql.session.timeZone", "UTC")


In [10]:
import pytz
print(pytz.all_timezones)



['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Asmera', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Timbuktu', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'Ameri

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_utc_timestamp, from_utc_timestamp

# Initialize Spark Session
spark = SparkSession.builder.appName("TimeZoneConversionExample").getOrCreate()

# Create Data
data = [
    (1, "2023-01-15 14:00:00", "America/New_York"),
    (2, "2023-01-15 11:00:00", "America/Los_Angeles"),
    (3, "2023-01-15 20:00:00", "Europe/London"),
    (4, "2023-01-16 03:00:00", "Asia/Tokyo"),
    (5, "2023-01-15 19:00:00", "UTC")
]

# Create DataFrame
df = spark.createDataFrame(data, ["event_id", "event_time", "time_zone"])

# Convert 'event_time' to Timestamp
df = df.withColumn("event_time", to_timestamp("event_time", "yyyy-MM-dd HH:mm:ss"))

# Convert to UTC
df_utc = df.withColumn("utc_time", to_utc_timestamp("event_time", "time_zone"))

# Convert to Specific Time Zone (e.g., America/New_York)
df_ny_time = df_utc.withColumn("ny_time", from_utc_timestamp("utc_time", "America/New_York"))

# Show Results
df_ny_time.show(truncate=False)



Py4JJavaError: An error occurred while calling o173.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 210) (localhost executor driver): java.time.zone.ZoneRulesException: Unknown time-zone ID: time_zone
	at java.base/java.time.zone.ZoneRulesProvider.getProvider(ZoneRulesProvider.java:279)
	at java.base/java.time.zone.ZoneRulesProvider.getRules(ZoneRulesProvider.java:234)
	at java.base/java.time.ZoneRegion.ofId(ZoneRegion.java:120)
	at java.base/java.time.ZoneId.of(ZoneId.java:408)
	at java.base/java.time.ZoneId.of(ZoneId.java:356)
	at java.base/java.time.ZoneId.of(ZoneId.java:312)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils$.getZoneId(DateTimeUtils.scala:61)
	at org.apache.spark.sql.catalyst.util.DateTimeUtils.getZoneId(DateTimeUtils.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.init(Unknown S

In [7]:
res3 = spark.sql(""" 
   SELECT e1.event_id AS event1_id, e2.event_id AS event2_id, 
       e1.event_time AS event1_time, e2.event_time AS event2_time, 
       CASE WHEN to_utc_timestamp(e1.event_time, e1.time_zone) = to_utc_timestamp(e2.event_time, e2.time_zone)
            THEN 'Same Time' ELSE 'Different Time' END AS comparison
FROM events e1
JOIN events e2 ON e1.event_id < e2.event_id;
              
                 """)

res3.show()

+---------+---------+-------------------+-------------------+--------------+
|event1_id|event2_id|        event1_time|        event2_time|    comparison|
+---------+---------+-------------------+-------------------+--------------+
|        1|        2|2023-01-15 14:00:00|2023-01-15 11:00:00|     Same Time|
|        1|        3|2023-01-15 14:00:00|2023-01-15 20:00:00|Different Time|
|        1|        4|2023-01-15 14:00:00|2023-01-16 03:00:00|Different Time|
|        1|        5|2023-01-15 14:00:00|2023-01-15 19:00:00|     Same Time|
|        1|        6|2023-01-15 14:00:00|2023-01-15 08:00:00|Different Time|
|        1|        7|2023-01-15 14:00:00|2023-01-15 21:00:00|Different Time|
|        1|        8|2023-01-15 14:00:00|2023-01-16 00:00:00|Different Time|
|        1|        9|2023-01-15 14:00:00|2023-01-15 13:30:00|Different Time|
|        1|       10|2023-01-15 14:00:00|2023-01-15 17:00:00|Different Time|
|        2|        3|2023-01-15 11:00:00|2023-01-15 20:00:00|Different Time|

In [ ]:
# Self-join to compare times
df_self_join = df.alias("e1").join(
    df.alias("e2"),
    col("e1.event_id") < col("e2.event_id")
).withColumn("e1_utc", to_utc_timestamp(col("e1.event_time"), col("e1.time_zone"))) \
 .withColumn("e2_utc", to_utc_timestamp(col("e2.event_time"), col("e2.time_zone"))) \
 .withColumn("comparison", 
             when(col("e1_utc") == col("e2_utc"), "Same Time").otherwise("Different Time"))

# Show the comparison result
df_self_join.select("e1.event_id", "e2.event_id", "e1.event_time", "e2.event_time", "comparison").show(truncate=False)
